
## Multiclass Logistic Regression

    

In [2]:
## Main Libraries

import numpy as np
from numpy.random import default_rng

from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax
from tqdm import tqdm

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row
from bokeh.palettes import Spectral11

output_notebook()
rng = default_rng()

Loading BokehJS ...

### The digits dataset

In this notebook we will use gradient descent and logistic regression to classify handwritten digits.

The digits dataset provided by sklearn consist of just under 1800 samples, each of which is an 8x8 image
with the pixels on a 0 to 10 scale. To recover the image, you must convert the 64 entry rows of the data matrix
into an 8x8 array. First we load the data (with the labels) and illustrate a few of the entries.


In [3]:
from sklearn import datasets

data = datasets.load_digits()["data"]
labels = datasets.load_digits()["target"]
samples = rng.choice(list(range(data.shape[0])), size=10)


def pic(samples):
    """Return renderings of data rows with indices from array samples"""
    L = []
    for i in samples:
        p = figure(
            height=80,
            width=80,
            match_aspect=True,
            toolbar_location=None,
            title="Label: {}".format(labels[i]),
        )
        p.grid.visible = False
        p.axis.visible = False
        k0 = np.flip(-data[i, :].reshape(8, 8), axis=0)
        p.image(image=[k0], x=0, y=0, dw=8, dh=8)
        L.append(p)
    x = row(L)
    return x


show(pic(samples))


### The computation

We add a column of ones to the data and we scale the data so it lies between 0 and 1 (the pixel values are between 0 and 10). This helps with numerical stability.

We use a library routine `OneHotEncoder` to convert the labels into a one-hot array.


In [4]:
data = np.concatenate([data, np.ones((data.shape[0], 1))], axis=1)
# scale the data
data = data / np.max(data)

# Use sklearn to create a "one-hot" encoding of the labels

E = OneHotEncoder()
Y = E.fit_transform(labels.reshape(-1, 1)).toarray()


In [5]:
## This is our main gradient descent routine. It also returns the sequence of computed likelihoods
## that we can plot to observe how our algorithm is performing


def descent(x, y, max_iter=1000, nu=0.001, M=None):
    """does gradient descent to maximum likelihood for data=x and labels (one-hot)=y"""
    features = x.shape[1]
    classes = y.shape[1]
    likes = []
    if M is None:
        M = rng.normal(loc=0, scale=1, size=(features, classes))
    for i in tqdm(range(max_iter)):
        P = softmax(x @ M, axis=1)
        grad = x.transpose() @ (P - y)
        M = M - nu * grad
        likes.append(-np.trace(y.transpose() @ np.log(P)))
    return M, likes


We call the descent algorithm with the data and the labels


In [6]:
M, L = descent(data, Y)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1559.37it/s]


To see how accurate we are, we compute the predicted labels by taking the column of the largest probability in
each row of the matrix P=sigma(XM) and compare it to the true labels.


In [7]:
predicted = np.argmax(softmax(data @ M, axis=1), axis=1)
correct = (predicted == labels).sum()
percentage = correct / len(labels)
print("Percentage correct on training data = {}".format(100 * percentage))


Percentage correct on training data = 98.60879243183082


In [8]:
p = figure(title="Negative Log Likelihood over iterations")
p.line(x=list(range(len(L))), y=L)
show(p)


## Batch Descent

To illustrate batch descent, we will break up our data matrix into 20 blocks of roughly 90 rows per block.
To avoid bias we will randomly shuffle the rows before breaking it up.


In [9]:
def block_descent(x, y, max_epochs=1000, block_size=50, nu=0.001, M=None):
    """does batch gradient descent to maximum likelihood for data=x and labels (one-hot)=y"""
    features = x.shape[1]
    classes = y.shape[1]
    likes = []
    blocks_per_epoch=x.shape[0]//block_size
    if M is None:
        M = rng.normal(loc=0, scale=1, size=(features, classes))
    for i in tqdm(range(max_epochs)):
        start=0
        for j in range(blocks_per_epoch):
            P = softmax(x[start:start+block_size,:] @ M, axis=1)
            grad = x[start:start+block_size,:].transpose() @ (P - y[start:start+block_size,:])
            M = M - nu * grad
            start = start+block_size
        P=softmax(x[start:,:] @ M, axis=1)
        grad = x[start:,:].transpose() @ (P-y[start:start+block_size,:])
        M = M-nu*grad
        likes.append(-np.trace(y.transpose() @ np.log(softmax(x @ M, axis=1))))
    return M, likes

In [10]:
M,L=block_descent(data,Y)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 362.59it/s]


In [11]:
predicted = np.argmax(softmax(data @ M, axis=1), axis=1)
correct = (predicted == labels).sum()
percentage = correct / len(labels)
print("Percentage correct on training data = {}".format(100 * percentage))

Percentage correct on training data = 98.33055091819699


In [12]:
p = figure(title="Negative Log Likelihood over iterations")
p.line(x=list(range(len(L))), y=L)
show(p)

## Stochastic Gradient Descent

The SGD algorithm is "batch descent" with a batch size of one. 

In [ ]:
M, L = block_descent(data,Y,block_size=1,max_epochs=500)

 62%|██████████████████████████████████████████████████▌                              | 312/500 [00:22<00:13, 14.41it/s]

In [13]:
predicted = np.argmax(softmax(data @ M, axis=1), axis=1)
correct = (predicted == labels).sum()
percentage = correct / len(labels)
print("Percentage correct on training data = {}".format(100 * percentage))

Percentage correct on training data = 97.9966611018364
